In [1]:
import sys
sys.path.append('..')

import torch
from torch import nn, Tensor
import torch.nn.functional as F

from dataset import sequence

import numpy as np
import matplotlib.pyplot as plt

from models.transformer import *

In [2]:
def train(model: nn.Module, optim: torch.optim, x: Tensor, t:Tensor, batch_size: int, max_epoch: int) -> list:
    loss_list = []
    
    model.train()
    for epoch in range(max_epoch):
        idx = np.random.permutation(np.arange(data_size))
        x = x[idx]
        t = t[idx]

        for iters in range(len(x) // batch_size):
            batch_x = x[(iters * batch_size) : (iters + 1) * batch_size]
            batch_t = t[(iters * batch_size) : (iters + 1) * batch_size]

            batch_x = torch.Tensor(batch_x).type(torch.long)
            batch_t = torch.Tensor(batch_t).type(torch.long)

            target = batch_t[:, 1:]
            batch_t = batch_t[:, :-1]

            mask = model.get_seq_mask(batch_t.shape[1])

            output = model.forward(batch_x, batch_t, mask)
            loss = criterion(output.view(-1, ntokens), target.reshape(-1))

            optim.zero_grad()
            loss.backward()
            optim.step()
            
            loss_list += [loss.item()]
            
            if iters % 20 == 0:
                print(f'{epoch}({iters}): {loss.item()}')
            
    return loss_list

In [3]:
def evaluate(model: nn.Module, optim: torch.optim, x: Tensor, t:Tensor, batch_size: int) -> list:
    loss_list = []
    
    model.eval()
    for iters in range(len(x) // batch_size):
        batch_x = x[(iters * batch_size) : (iters + 1) * batch_size]
        batch_t = t[(iters * batch_size) : (iters + 1) * batch_size]
        
        batch_x = torch.Tensor(batch_x).type(torch.long)
        batch_t = torch.Tensor(batch_t).type(torch.long)
        
        target = batch_t[:, 1:]
        batch_t = batch_t[:, :-1]
        
        mask = model.get_seq_mask(batch_t.shape[1])
        
        output = model.forward(batch_x, batch_t, mask)
        loss = criterion(output.view(-1, ntokens), target.reshape(-1))
        
        loss_list += [loss.item()]
        
        if iters % 20 == 0:
            print(f'{iters}: {loss.item()}')
            
    return loss_list

In [4]:
def generate(model: nn.Module, x: Tensor, t:Tensor, token_id: int, seq_len: int):
    b, n = x.shape
    x = torch.Tensor(x).type(torch.long)
    mask = model.get_seq_mask(seq_len)
    pred = model.generate(x, torch.Tensor([token_id]).repeat(b).unsqueeze(-1).type(torch.long), seq_len, mask)
    
    
    for b in range(b):
        query = ''.join(id_to_char[x] for x in x[b].numpy())
        target = ''.join(id_to_char[x] for x in t[b])
        a = ''.join(id_to_char[x] for x in pred[b].numpy())

        print(f'Q: {query}')
        print(f'T: {target}')
        print(f'{"O" if target == a else "X"}: {a}')
        print('--')

In [5]:
def loss_graph(loss_list: list, label: str):
    iters = np.arange(len(train_loss))
    plt.plot(iters, train_loss, label='train')
    plt.xlabel('iters')
    plt.ylabel(label)
    plt.show()

In [6]:
(x_train, t_train), (x_test, t_test) = sequence.load_data('date.txt')
char_to_id, id_to_char = sequence.get_vocab()

In [7]:
ntokens = len(char_to_id)
max_len = 100
d_model = 128
d_ff = d_model * 4
nheads = 2
nlayers = 2
max_len = 100
seq_len = t_train.shape[1] - 1
learning_rate = 6e-4

batch_size = 64
max_epoch = 3
data_size = len(x_train)

In [8]:
model = Transformer(max_len, ntokens, d_model, nheads, d_ff, nlayers)
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
train_loss = train(model, optim, x_train, t_train, batch_size, max_epoch)

0(0): 61.17148971557617
0(20): 3.2936928272247314
0(40): 1.9102089405059814
0(60): 1.7076330184936523
0(80): 1.5900615453720093
0(100): 1.5406310558319092
0(120): 1.5421806573867798
0(140): 1.540207862854004
0(160): 1.48826265335083
0(180): 1.5049744844436646
0(200): 1.456982135772705
0(220): 1.481579065322876
0(240): 1.4611189365386963
0(260): 1.504856824874878
0(280): 1.4266360998153687
0(300): 1.4146411418914795
0(320): 1.3832087516784668
0(340): 1.34346604347229
0(360): 1.3736015558242798
0(380): 1.323213815689087
0(400): 1.3603177070617676
0(420): 1.3559712171554565
0(440): 1.2796876430511475
0(460): 1.2747551202774048
0(480): 1.1789920330047607
0(500): 1.0766452550888062
0(520): 1.0884976387023926
0(540): 1.006840467453003
0(560): 0.9062819480895996
0(580): 0.9025230407714844
0(600): 0.7783195972442627
0(620): 0.6859427690505981
0(640): 0.7368806600570679
0(660): 0.6920230984687805
0(680): 0.7519000172615051
0(700): 0.6632137298583984
1(0): 0.6843469738960266
1(20): 0.63961321115

In [ ]:
test_loss = evaluate(model, optim, x_train, t_train, batch_size)

In [ ]:
loss_graph(train_loss, 'train loss')

In [ ]:
generate(model, x_test[:10], t_test[:10, 1:], 14, 10)